In [ ]:
from  TDClient import FData
import time
import pandas as pd
import datetime
import pytz
indian_tz = pytz.timezone('Asia/Kolkata')
import warnings
warnings.filterwarnings("ignore")

username = 'tdwsp353'
password = 'anjan@353'

nurl = 'http://3.6.166.159/td'                 
url = "http://127.0.0.1:5000"
#nurl='http://192.168.2.173:6969'

td_obj = FData(key='trial', base_url=nurl)
td_obj.login()
td_obj.connect_to_server(username=username, password=password, realtime_port=8082, url='push.truedata.in')

list_of_symbols = ['NIFTY','BANKNIFTY']

year,month,day,length = 2023,6,29,8

for sym in list_of_symbols:
    td_obj.add_option_symbols(sym, year, month, day,length=length)
time.sleep(5)
data = td_obj.get_oc_data()
syms = list(data.keys())
print(f"Downloaded data for {[print(x) for x in syms]}")

df_dict = {k:pd.DataFrame.from_records(v) for k,v in data.items()}
df_dict['BANKNIFTY2023629']

In [ ]:
from sqlalchemy.sql.expression import true

sym = 'NIFTY'
year,month,day,length = 2023,7,27,30

start_time = datetime.time(9, 15)
end_time = datetime.time(14, 50)
current_time = datetime.datetime.now(indian_tz).time()

trade_entry = []
trade_exits = []
live_pnl = []
profit = []
hedge_profit = []

trade = False

In [ ]:
td_obj.add_option_symbols(sym, year, month, day,length)
time.sleep(5)

In [ ]:
while (current_time>start_time)&(current_time<end_time):
    
    try:
    
        df = pd.DataFrame(td_obj.get_oc_data()[sym + str(year) + str(month)+ str(day)])
        df = df[['ltp', 'ask','bid', 'ltt', 'strike', 'type']]
        df.index = df['strike']
        df= df.drop(columns =['strike'])
        df.ltt = pd.to_datetime(df.ltt)
        df = df[(df['ask']!=0)&(df['bid']!=0)]

        df_ce = df[df['type'] == 'CE']
        df_pe = df[df['type'] == 'PE']

        diff_df = abs(df_ce['bid'] - df_pe['bid']).sort_values()

        if trade == False:
            print(current_time,diff_df[0])

        if (diff_df[0]<5) & (trade == False):

            ce_premium = df_ce[df_ce.index==diff_df.index[0]]['bid'][0]
            pe_premium = df_pe[df_pe.index==diff_df.index[0]]['bid'][0]

            straddle_strike = diff_df.index[0]
            straddle_premium = ce_premium + pe_premium

            hedge_budget = straddle_premium*0.30

            df_ce['dist_from_strike'] = int(straddle_strike) - df_ce.index.astype(int)
            df_pe['dist_from_strike'] = df_pe.index.astype(int) - int(straddle_strike)

            df_ce['strike_price'] = df_ce.index
            df_pe['strike_price'] = df_pe.index

            df_ce.index = df_ce['dist_from_strike']
            df_pe.index = df_pe['dist_from_strike']

            df_ce = df_ce.rename(columns = {'strike_price':'ce_strike_price', 'ask':'ce_ask', 'bid':'ce_bid'})
            df_pe = df_pe.rename(columns = {'strike_price':'pe_strike_price', 'ask':'pe_ask', 'bid':'pe_bid'})

            df_hedge = pd.concat([df_ce,df_pe], axis = 1)
            df_hedge['hedge_premium'] = df_hedge['ce_ask'] + df_hedge['pe_ask']
            df_hedge = df_hedge.dropna()
            df_hedge['closest_to_budget'] = df_hedge['hedge_premium'] - hedge_budget
            df_hedge = df_hedge[df_hedge['closest_to_budget']<0]
            df_hedge = df_hedge.sort_values(by = 'closest_to_budget', ascending = False)

            ce_hedge_strike = df_hedge['ce_strike_price'].iloc[0]
            ce_hedge_premium = df_hedge['ce_ask'].iloc[0]
            pe_hedge_strike = df_hedge['pe_strike_price'].iloc[0]
            pe_hedge_premium = df_hedge['pe_ask'].iloc[0]
            hedge_premium = ce_hedge_premium + pe_hedge_premium

            if hedge_premium < hedge_budget:

                trade = True

                trade_entry.append([current_time,
                                'Buy',
                                'Straddle Strike Price :',
                                straddle_strike,
                                'Straddle Premium:',
                                straddle_premium,
                                'Hedge CE Strike:',
                                ce_hedge_strike,
                                'Hedge PE Strike:',
                                pe_hedge_strike,
                                'Hedge Premium:',
                                hedge_premium
                                ])

                print(trade_entry[-1])

        if trade == True:

            difference_ce_pe = abs(df.loc[straddle_strike]['bid'][0] - df.loc[straddle_strike]['bid'][1])

            sell_straddle_premium = df.loc[straddle_strike]['ask'].sum()

            live_profits = (straddle_premium - sell_straddle_premium)*50

            live_pnl.append([current_time,
                             'Trade Number:',
                             len(trade_entry),
                             'Difference CE & PE:'+ str(round(difference_ce_pe,2)),
                             'Live PnL of Straddle:' + str(round(live_profits,2))
                             ])

            print(live_pnl[-1])

        if trade == True:

            if (abs(df.loc[straddle_strike]['bid'][0] - df.loc[straddle_strike]['bid'][1])>75):

                sell_straddle_premium = df.loc[straddle_strike]['ask'].sum()

                ceh1 = df.loc[ce_hedge_strike]
                ceh1 = ceh1[ceh1['type'] == 'CE']['bid'][0]

                peh1 = df.loc[pe_hedge_strike]
                peh1 = peh1[peh1['type'] == 'PE']['bid'][0]

                sell_hedge_premium = ceh1 + peh1

                trade_exits.append([current_time,
                                    'Sell',
                                    'Straddle Sell Premium:' + str(sell_straddle_premium),
                                    'Hedge Sell Premium:' + str(sell_hedge_premium)
                                    ])

                pf = (straddle_premium - sell_straddle_premium)*50

                profit.append([current_time,
                               'Profit :',
                               pf])

                trade = False

                print(trade_exits[-1])
                print(profit[-1])
                print('*' * 126)
                print("\n")
                
        if len(trade_entry)>=1:
            
            ce_hedge_strike = trade_entry[0][7]
            pe_hedge_strike = trade_entry[0][9]

            ceh1 = df.loc[ce_hedge_strike]
            ceh1 = ceh1[ceh1['type'] == 'CE']['bid'][0]

            peh1 = df.loc[pe_hedge_strike]
            peh1 = peh1[peh1['type'] == 'PE']['bid'][0]

            sell_hedge_premium = ceh1 + peh1

            hedge_premium = trade_entry[0][-1]
            
            hpf = (sell_hedge_premium-hedge_premium)*25
            
            print(current_time,'Hedge Live Profits: ', hpf)
            print("\n")

        if trade == True:

            if current_time>=datetime.time(14,49):

                sell_straddle_premium = df.loc[straddle_strike]['ask'].sum()

                ce_hedge_strike = trade_entry[0][7]
                pe_hedge_strike = trade_entry[0][9]

                ceh1 = df.loc[ce_hedge_strike]
                ceh1 = ceh1[ceh1['type'] == 'CE']['bid'][0]

                peh1 = df.loc[pe_hedge_strike]
                peh1 = peh1[peh1['type'] == 'PE']['bid'][0]

                sell_hedge_premium = ceh1 + peh1
                
                hedge_premium = trade_entry[0][-1]

                trade_exits.append([current_time,
                                    'Sell',
                                    'Straddle Sell Premium:' + str(sell_straddle_premium),
                                    'Hedge Sell Premium:' + str(sell_hedge_premium)
                                    ])

                pf = (straddle_premium - sell_straddle_premium)*50

                profit.append([current_time,
                               'Profit :',
                               pf])

                hedge_profit.append([current_time,
                                   'Profit :',
                                    (sell_hedge_premium-hedge_premium)*50,
                                    ])

                trade = False

                print(trade_exits[-1])
                print(profit[-1])
                print('*' * 125)
                print("\n")

                break
        
    except Exception as e:
        
        td_obj = FData(key='trial', base_url=nurl)
        td_obj.login()
        td_obj.connect_to_server(username=username, password=password, realtime_port=8082, url='push.truedata.in')
        
        td_obj.add_option_symbols(sym, year, month, day,length)
        time.sleep(5)
        
        print("Restart" + ' '+str(e))
        pass
    
    time.sleep(10)
    current_time = datetime.datetime.now(indian_tz).time()

In [ ]:
profit

In [ ]:
hedge_profit

In [ ]:
print('Nifty', 'Profit :', 
      pd.concat([pd.DataFrame(profit), pd.DataFrame(hedge_profit)], axis = 0)[2].sum(),
      '\n Shiftings : ', 
      len(profit))